# Setup/Test Environment:

In [1]:
from web_footprinting_utils import setup_ipython
import os

get_ipython, INTERACTIVE = setup_ipython()

In [2]:
import sys
sys.executable

'/root/miniconda/bin/python'

In [3]:
%env TARGET=10.10.129.98

env: TARGET=10.10.129.98


In [4]:
!echo Make sure we are connected to VPN:
result = !ip a | grep tun0

if not len(result)>0:
    print("Not connected!")
#     input()
else:
    print("Connected!")
    print(result)

Make sure we are connected to VPN:
Not connected!


# Initial (Unreliable) RustScan:

In [5]:
import os
pwd = os.getcwd()
rustscan = "docker run -v '%s':/workspace  --volume /etc/passwd:/etc/passwd:ro --volume /etc/group:/etc/group:ro --user $(id -u) --rm my-other-github-account/rustscan" % pwd

In [ ]:
!{rustscan} -t 4000 -r 1-65535 -a {os.environ["TARGET"]} -- -Pn -A -sV

.----. .-. .-. .----..---.  .----. .---.   .--.  .-. .-.
| {}  }| { } |{ {__ {_   _}{ {__  /  ___} / {} \ |  `| |
| .-. \| {_} |.-._} } | |  .-._} }\     }/  /\  \| |\  |
`-' `-'`-----'`----'  `-'  `----'  `---' `-'  `-'`-' `-'
The Modern Day Port Scanner.
________________________________________
: https://discord.gg/GFrQsGy           :
: https://github.com/RustScan/RustScan :
 --------------------------------------
Real hackers hack time ⌛

[~] The config file is expected to be at "/root/.rustscan.toml"
[~] File limit higher than batch size. Can increase speed by increasing batch size '-b 1048476'.


# Initial Nmap:

In [ ]:
!echo $TARGET
!mkdir -p scans/quick scans/tcp scans/udp scans/scripts scans/http scans/no_ping

In [ ]:
!ping -c 5 $TARGET

# Quick Scan:

In [ ]:
!nmap -vv -oA scans/quick/quick_scan $TARGET

# All Ports:

In [ ]:
!nmap -p- -sV -vv -oA  scans/tcp/tcp_scan $TARGET

In [ ]:
import xmltodict

with open('scans/tcp/tcp_scan.xml','r') as f:
    xml_dict = xmltodict.parse(f.read())

In [ ]:
PING=True
try:
    port_list = [port['@portid'] for port in xml_dict['nmaprun']['host']['ports']['port']]
    %env PORTS={",".join(port_list)}
except:
    print("Host is down")
    PING=False

# No Ping:

In [ ]:
!nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET

In [ ]:
if not PING:
    !nmap -sV -Pn -p- -vv -oA  scans/tcp/tcp_scan $TARGET

# Run Default Scripts:

In [ ]:
if PING:
    !nmap -sC -p $PORTS -vv -oA scans/scripts/script_scan $TARGET

In [ ]:
if PING:
    !nmap -A -T5 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T5 -Pn -vv --script=default,safe,discovery $TARGET

In [ ]:
if PING:
    !nmap -A -T3 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T3 -Pn -vv --script=default,safe,discovery $TARGET

In [ ]:
from getpass import getpass
import os

if PING:
    !echo { getpass() } | sudo -S nmap -A -p- -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}
else:
    !echo { getpass() } | sudo -S nmap -A -p- -Pn -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}

In [ ]:
# UDP Scan, this takes forever...

# from getpass import getpass

# !echo { getpass() } > tmp.txt
# !cat tmp.txt | sudo -S nmap -sU --open -oA scans/udp/udp_scan $TARGET
# !cat tmp.txt | sudo -S nmap -sU --open -p- -oA scans/udp/udp_scan $TARGET
# !rm -rf tmp.txt